# Aplicando a Spark engine no watsonx.data através da API

### Configuração das credenciais

Esta célula define as credenciais para acessar o watsonx.data. Na próxima seção mostraremos como capturá -las. As credenciais incluem:

wxd_hms_endpoint: o endpoint do serviço watsonx.data

wxd_hms_username: o nome de usuário para acessar o watsonx.data

wxd_hms_password: a senha para acessar o watsonx.data

source_bucket_endpoint: o endpoint do bucket de armazenamento de dados

source_bucket_access_key: a chave de acesso para o bucket de armazenamento de dados

source_bucket_secret_key: a chave secreta para o bucket de armazenamento de dados

In [ ]:
wxd_hms_endpoint = "thrift://aaaaaaaa-bbbb-cccc-dddd-abcdef012345.abcdefghijk0123456789.lakehouse.appdomain.cloud:12345"
wxd_hms_username = "ibmlhapikey"
wxd_hms_password = "wxd_hms_password"
source_bucket_endpoint = "s3.br-sao.cloud-object-storage.appdomain.cloud"
source_bucket_access_key = "your_source_access_key"
source_bucket_secret_key = "source_bucket_secret_key"

## Coletando as credenciais

Agora vamos realizar a configuração da conexão com o watsonx.data. Para isso, navegue até a aba de Infrastructure Manager, onde você poderá ver todas as conexões entre engines, catálogos e storages. 

Para configurar a conexão com a engine escolhida, clique na opção correspondente para abrir uma nova janela com detalhes adicionais das credenciais. Essa janela fornecerá as informações necessárias para estabelecer a conexão segura e eficaz com o watsonx.data.

<img src="images/wxd-1.png" alt="wxd-1"  />

Uma vez aberta a janela, procure pela aba de "Visualizar configurações" para de fato exibir todas as credenciais necessárias para continuar o fluxo de configuração.

<img src="images/wxd-2.png" alt="wxd2"  />

<img src="images/wxd-3.png" alt="wxd3"  />

Agora que você tem as configurações em mãos, é hora de adaptá-las às suas credenciais específicas do Watsonx.data e Cloud Object Storage.

Com essas informações em mãos, você pode adaptar as configurações para as suas credenciais específicas. Certifique-se de substituir os valores padrão pelas suas credenciais reais.

Lembre-se de que é importante manter suas credenciais seguras e não compartilhá-las com ninguém. Além disso, certifique-se de que você está usando as credenciais corretas para o seu watsonx.data e Cloud Object Storage.

In [ ]:
# Configuração Spark para wxd
conf=spark.sparkContext.getConf()
spark.stop()

from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession


conf.setAll([("spark.sql.catalogImplementation", "hive"), \
    ("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"), \
    ("spark.sql.iceberg.vectorization.enabled", "false"), \
    ("spark.sql.catalog.lakehouse", "org.apache.iceberg.spark.SparkCatalog"), \
    ("spark.sql.catalog.lakehouse.type", "hive"), \
    ("spark.sql.catalog.lakehouse.uri", wxd_hms_endpoint), \
    ("spark.hive.metastore.client.auth.mode", "PLAIN"), \
    ("spark.hive.metastore.client.plain.username", wxd_hms_username), \
    ("spark.hive.metastore.client.plain.password", wxd_hms_password), \
    ("spark.hive.metastore.use.SSL", "true"), \
    ("spark.hive.metastore.truststore.type", "JKS"), \
    ("spark.hive.metastore.truststore.path", "file:///opt/ibm/jdk/lib/security/cacerts"), \
    ("spark.hive.metastore.truststore.password", "changeit"), \
    ("spark.hadoop.fs.s3a.bucket.wxd-silver.endpoint", source_bucket_endpoint), \
    ("spark.hadoop.fs.s3a.bucket.wxd-silver.access.key", source_bucket_access_key), \
    ("spark.hadoop.fs.s3a.bucket.wxd-silver.secret.key", source_bucket_secret_key), \
])

spark=SparkSession.builder.config(conf=conf).getOrCreate()

>>
### Listando os databases watsonx.data


Uma vez que temos a conexão pronta, podemos listar os bancos de dados existentes no watsonx.data

In [ ]:
def list_databases(spark):
    spark.sql("show databases from lakehouse").show()

In [ ]:
list_databases(spark)

In [ ]:
spark.sql("show tables from lakehouse.teste_silver").show()

Na célula abaixo, estamos executando uma consulta em Spark para recuperar dados do lakehouse, transformando a consulta para Pandas e medindo o tempo de execução da consulta.

Isso nos permite entender como o Spark está lidando com os dados e identificar possíveis otimizações.



In [ ]:
start_time = time.time()

df_spark = spark.sql("""
SELECT 
  *
FROM
  lakehouse.teste_silver.cadastro
""").toPandas()

df_spark_time = time.time() - start_time

df_spark.head()